In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df = df.drop_duplicates()
df['sex'] = pd.factorize(df['sex'])[0] + 1
df['region'] = pd.factorize(df['region'])[0] + 1
df['smoker'] = pd.factorize(df['smoker'])[0] + 1
corr = df.corr()
corr['charges'].sort_values(ascending=False)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


charges     1.000000
age         0.298308
bmi         0.198401
children    0.067389
sex         0.058044
region      0.006547
smoker     -0.787234
Name: charges, dtype: float64

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
X = df.drop('charges', axis = 1)
y = df['charges']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=101)
# применим стандартизацию
scaler= StandardScaler()
scaler.fit(X_train)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_train_scaled

array([[ 0.91437827, -1.00321372,  1.25713624,  0.75514165,  0.53488998,
         1.41354968],
       [-0.22687811,  0.99679657,  0.14285056, -0.92394858,  0.53488998,
        -1.35144813],
       [ 0.84304975, -1.00321372,  1.43853158, -0.08440347,  0.53488998,
        -1.35144813],
       ...,
       [ 1.27102089,  0.99679657,  0.54937048, -0.92394858,  0.53488998,
         0.49188375],
       [-1.51079153, -1.00321372,  1.38346514, -0.92394858,  0.53488998,
        -0.42978219],
       [ 0.84304975,  0.99679657, -0.84510621, -0.92394858,  0.53488998,
        -1.35144813]])

In [4]:
# введем ошибку
def scores(y, predictions):
    mae = mean_absolute_error(y, predictions)
    mse = mean_squared_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    r_squared = r2_score(y, predictions)
    return mae, mse, rmse, r_squared

models = pd.DataFrame(columns=["Model","MAE","MSE","RMSE","R2 Score"])

# 2. Градиентный бустинг

In [7]:
Gradient_model = GradientBoostingRegressor()
Gradient_model.fit(X_train_scaled, y_train)

GradientBoostingRegressor()

In [8]:
y_pred = Gradient_model.predict(X_test_scaled)
y_pred = pd.DataFrame(y_pred)
MAE_gradient= metrics.mean_absolute_error(y_test, y_pred)
MSE_gradient = metrics.mean_squared_error(y_test, y_pred)
RMSE_gradient =np.sqrt(MSE_gradient)
pd.DataFrame([MAE_gradient, MSE_gradient, RMSE_gradient], index=['MAE_gradient', 'MSE_gradient', 'RMSE_gradient'], columns=['Metrics'])

,Metrics
MAE_gradient,2.542526e+03
MSE_gradient,2.222673e+07
RMSE_gradient,4.714524e+03


In [9]:
scores = cross_val_score(Gradient_model, X_train_scaled, y_train, cv=5)
print(np.sqrt(scores))

[0.93656555 0.89843925 0.92393087 0.94831519 0.92925657]


In [12]:
r2_score(y_test, Gradient_model.predict(X_test_scaled))

0.8413923963001929

# Решающие дерево

In [14]:
tree_reg_model =DecisionTreeRegressor()
tree_reg_model.fit(X_train_scaled, y_train);

In [15]:
y_pred = tree_reg_model.predict(X_test_scaled)
y_pred = pd.DataFrame(y_pred)
MAE_tree_reg= metrics.mean_absolute_error(y_test, y_pred)
MSE_tree_reg = metrics.mean_squared_error(y_test, y_pred)
RMSE_tree_reg =np.sqrt(MSE_tree_reg)
pd.DataFrame([MAE_tree_reg, MSE_tree_reg, RMSE_tree_reg], index=['MAE_tree_reg', 'MSE_tree_reg', 'RMSE_tree_reg'], columns=['Metrics'])

,Metrics
MAE_tree_reg,3.003140e+03
MSE_tree_reg,4.076725e+07
RMSE_tree_reg,6.384923e+03


In [16]:
scores = cross_val_score(tree_reg_model, X_train_scaled, y_train, cv=5)
print(np.sqrt(scores))

[0.89479472 0.79464473 0.85930552 0.7953075  0.81046331]


In [17]:
r2_score(y_test, tree_reg_model.predict(X_test_scaled))

0.7090892591842208

# Случайный лес

In [18]:
forest_reg_model =RandomForestRegressor()
forest_reg_model.fit(X_train_scaled, y_train);

In [19]:
y_pred = forest_reg_model.predict(X_test_scaled)
y_pred = pd.DataFrame(y_pred)
MAE_forest_reg= metrics.mean_absolute_error(y_test, y_pred)
MSE_forest_reg = metrics.mean_squared_error(y_test, y_pred)
RMSE_forest_reg =np.sqrt(MSE_forest_reg)
pd.DataFrame([MAE_forest_reg, MSE_forest_reg, RMSE_forest_reg], index=['MAE_forest_reg', 'MSE_forest_reg', 'RMSE_forest_reg'], columns=['Metrics'])

,Metrics
MAE_forest_reg,2.647356e+03
MSE_forest_reg,2.431055e+07
RMSE_forest_reg,4.930573e+03


In [20]:
scores = cross_val_score(forest_reg_model, X_train_scaled, y_train, cv=5)
print(np.sqrt(scores))

[0.93744146 0.88935521 0.91896247 0.92617471 0.91379618]


In [21]:
r2_score(y_test, forest_reg_model.predict(X_test_scaled))

0.8265225297000116

# Результаты

Большинство людей здесь молодого возраста (от 18 до 24 лет)

50,5% - мужчины и 49,5% - женщины

Людей без детей больше

79,5% - некурящие и 20,5% - курящие

В юго-восточном регионе больше всего людей

Сборы для курильщиков выше, чем для некурящих

в регионах заряженных мужчин намного выше, чем женщины